In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

In [ ]:
# Load the XML file
tree = ET.parse('CMT06.xml')
root = tree.getroot()

In [ ]:
# Extract the node and edge data from the XML file
node_list = []
demand_list = []
for node in root.findall("./network/nodes/"):
    node_id = int(node.attrib['id'])
    node_type = int(node.attrib['type'])
    node_x = float(node.find('cx').text)
    node_y = float(node.find('cy').text)
    node_list.append([node_id, node_type, node_x, node_y])
for demand in root.findall("./requests/"):
    node_id = int(demand.attrib['node'])
    node_demand = float(demand.find('quantity').text)
    demand_list.append([node_id, node_demand])

In [ ]:
# Convert the node_list and edge_list into dataframes
node_df = pd.DataFrame(node_list, columns=["id", "type", "x", "y"])
demand_df = pd.DataFrame(demand_list, columns=["id", "demand"])

In [ ]:
node_df.tail()

,id,type,x,y
46,47,1,25.0,32.0
47,48,1,25.0,55.0
48,49,1,48.0,28.0
49,50,1,56.0,37.0
50,51,0,30.0,40.0


In [ ]:
demand_df.tail()

,id,demand
45,46,5.0
46,47,25.0
47,48,17.0
48,49,18.0
49,50,10.0


In [ ]:
f=len(demand_df)

In [ ]:
from scipy.spatial import distance_matrix

In [ ]:
# Extract the x and y coordinates as a numpy array
coords = node_df[['x', 'y']].to_numpy()

# Calculate the distance matrix using the Euclidean distance metric
dist_matrix = distance_matrix(coords, coords)

In [ ]:
dist_matrix

array([[ 0.        , 12.36931688, 19.20937271, ..., 26.40075756,
        24.20743687, 13.89244399],
       [12.36931688,  0.        , 15.29705854, ..., 21.02379604,
        13.89244399, 21.02379604],
       [19.20937271, 15.29705854,  0.        , ..., 36.22154055,
        27.29468813, 32.55764119],
       ...,
       [26.40075756, 21.02379604, 36.22154055, ...,  0.        ,
        12.04159458, 21.63330765],
       [24.20743687, 13.89244399, 27.29468813, ..., 12.04159458,
         0.        , 26.17250466],
       [13.89244399, 21.02379604, 32.55764119, ..., 21.63330765,
        26.17250466,  0.        ]])

In [ ]:
demand_df['demand'].sum()

777.0

In [ ]:
demand = list(demand_df['demand'])

In [ ]:
demand.append(0)

In [ ]:
import numpy as np
import random
random.seed(10)

def fitness(chromosome, distance_matrix, demands, vehicle_capacity, depot_idx, service_time, time_per_distance, max_duration):
    # Calculate the route and number of vehicles for the chromosome.
    route, num_vehicles = chromosome_to_route(chromosome, distance_matrix, demands, vehicle_capacity, depot_idx, service_time, time_per_distance, max_duration)

    # Calculate the total distance of the route.
    total_distance = 0
    for sub_route in route:
        for i in range(len(sub_route) - 1):
            total_distance += distance_matrix[sub_route[i]][sub_route[i + 1]]

    # Create a fitness score with a penalty for the number of vehicles used.
    # You can adjust the weight to change the importance of the number of vehicles relative to the total distance.
    weight = 1.0
    fitness = 1 / total_distance - weight * num_vehicles

    return fitness



def crossover(parent1, parent2):
    idx1, idx2 = sorted(random.sample(range(len(parent1)), 2))

    child1 = [-1] * len(parent1)
    child1[idx1:idx2] = parent1[idx1:idx2]

    child2 = [-1] * len(parent2)
    child2[idx1:idx2] = parent2[idx1:idx2]

    p2_index = idx2
    for i in range(len(parent2)):
        if parent2[p2_index % len(parent2)] not in child1:
            child1[child1.index(-1)] = parent2[p2_index % len(parent2)]
        p2_index += 1

    p1_index = idx2
    for i in range(len(parent1)):
        if parent1[p1_index % len(parent1)] not in child2:
            child2[child2.index(-1)] = parent1[p1_index % len(parent1)]
        p1_index += 1

    return [child1, child2]

'''def crossover(parent1, parent2):
    size = len(parent1)
    # Choose random start/end indices for crossover in parent1
    start, end = sorted(random.sample(range(size), 2))

    child1 = [-1]*size
    child2 = [-1]*size

    # Copy the segment from parent1 to child1 and from parent2 to child2
    child1[start:end] = parent1[start:end]
    child2[start:end] = parent2[start:end]

    # Fill the remaining position of child1 from parent2
    p2 = end
    c1 = end
    while -1 in child1:
        if parent2[p2] not in child1:
            child1[c1] = parent2[p2]
            c1 = (c1 + 1) % size
        p2 = (p2 + 1) % size

    # Fill the remaining position of child2 from parent1
    p1 = end
    c2 = end
    while -1 in child2:
        if parent1[p1] not in child2:
            child2[c2] = parent1[p1]
            c2 = (c2 + 1) % size
        p1 = (p1 + 1) % size

    return [child1, child2]'''




def mutate(chromosome):
    idx1, idx2 = random.sample(range(len(chromosome)), 2)
    chromosome[idx1], chromosome[idx2] = chromosome[idx2], chromosome[idx1]

def chromosome_to_route(chromosome, distance_matrix, demands, vehicle_capacity, depot_idx, service_time, time_per_distance, max_duration):
    route = []
    num_vehicles = 0
    current_vehicle_capacity = vehicle_capacity
    current_duration = 0
    current_load = 0
    current_route = [depot_idx]
    previous_customer = depot_idx

    for customer in chromosome:
        if customer == depot_idx:
            continue

        distance = distance_matrix[previous_customer][customer]
        service_duration = service_time
        distance2 = distance_matrix[customer][depot_idx]
        sum_dis = distance + distance2

        if (current_load + demands[customer] <= current_vehicle_capacity and
            current_duration + sum_dis * time_per_distance + service_duration <= max_duration):
            current_route.append(customer)
            current_load += demands[customer]
            current_duration += distance * time_per_distance + service_duration
        else:
            current_route.append(depot_idx)
            route.append(current_route)
            current_route = [depot_idx, customer]
            current_load = demands[customer]
            current_duration = distance_matrix[depot_idx][customer] * time_per_distance
            num_vehicles += 1

        previous_customer = customer

    current_route.append(depot_idx)
    route.append(current_route)
    num_vehicles += 1

    return route, num_vehicles



def tournament_selection(population, fitness_values, tournament_size):
    selected_indices = random.sample(range(len(population)), tournament_size)
    best_index = max(selected_indices, key=lambda i: fitness_values[i])
    return population[best_index]


def genetic_algorithm(distance_matrix, demands, depot_idx, vehicle_capacity, service_time, time_per_distance, max_duration, population_size=1000, generations=5000, mutation_rate=0.05, crossover_rate=0.9, tournament_size=15, elitism_rate=0.1):
    customer_count = len(distance_matrix) - 1

    def create_chromosome():
        excluded_number = depot_idx  # replace with the number you want to exclude
        sample_size = customer_count
        range_to_sample_from = [i for i in range(len(distance_matrix)) if i != excluded_number]
        random_sample = random.sample(range_to_sample_from, sample_size)
        return random_sample

    best_fitness = -np.inf
    fitness_count = 0

    population = [create_chromosome() for _ in range(population_size)]
    better_chromosome=[]

    for i in range(f):
      better_chromosome.append(i)

    for i in range(generations):
        fitness_values = [fitness(chrom, distance_matrix, demands, vehicle_capacity, depot_idx, service_time, time_per_distance, max_duration) for chrom in population]

        if max(fitness_values) > best_fitness:
            best_fitness = max(fitness_values)
            fitness_count = 0
        else:
            fitness_count += 1

        # Elitism: Select the top elitism_size individuals
        elitism_size = int(elitism_rate * population_size)
        elite_indices = sorted(range(len(fitness_values)), key=lambda i: fitness_values[i], reverse=True)[:elitism_size]
        elite_chromosomes = [population[i] for i in elite_indices]

        new_population = []
        for _ in range((population_size - elitism_size) // 2):
            parent1 = tournament_selection(population, fitness_values, tournament_size)
            parent2 = tournament_selection(population, fitness_values, tournament_size)
            if random.random() < crossover_rate:
                child1, child2 = crossover(parent1, parent2)

            if random.random() < mutation_rate:
                mutate(child1)
            if random.random() < mutation_rate:
                mutate(child2)

            new_population.extend([child1, child2])

        # Include the elite chromosomes in the new population
        new_population.extend(elite_chromosomes)
        population = new_population

        best_chromosome = max(population, key=lambda chrom: fitness(chrom, distance_matrix, demands, vehicle_capacity, depot_idx, service_time, time_per_distance, max_duration))
        if fitness(best_chromosome, distance_matrix, demands, vehicle_capacity, depot_idx, service_time, time_per_distance, max_duration) > fitness(better_chromosome, distance_matrix, demands, vehicle_capacity, depot_idx, service_time, time_per_distance, max_duration):
            better_chromosome = best_chromosome

        if (generation % 1000 == 0):
            print(f"Generations: {generation}")
            print(f"Best Fitness: {best_fitness}")

        if fitness_count >= 1000:
            print(f"Stopping at generation {generation} with best fitness {best_fitness}")
            break

    best_route = chromosome_to_route(better_chromosome, distance_matrix, demands, vehicle_capacity, depot_idx, service_time, time_per_distance, max_duration)
    return best_route




def print_solution(solution, distance_matrix, demands, depot_idx, service_time, time_per_distance):
    total_distance = 0
    total_load = 0
    total_time = 0

    routes = solution[0]  # Get the list of routes from the solution
    num_vehicles = solution[1]  # Get the number of vehicles from the solution

    for vehicle, route in enumerate(routes):
        distance = 0
        load = 0
        time = 0
        prev_node = depot_idx
        print(f"Route for vehicle {vehicle}:")

        for node in route[1:-1]:  # Exclude the first and last depot nodes
            load += demands[node]
            distance += distance_matrix[prev_node][node]
            time += (distance_matrix[prev_node][node] * time_per_distance) + service_time
            print(f" {prev_node} Load({load}) ->", end="")
            prev_node = node

        # Add the distance and time from the last node to the depot
        distance += distance_matrix[prev_node][depot_idx]
        time += distance_matrix[prev_node][depot_idx] * time_per_distance
        print(f" {depot_idx} Load({load})")
        print(f"Distance of the route: {distance}m")
        print(f"Load of the route: {load}")
        print(f"Time of the route: {time - service_time}")

        total_distance += distance
        total_load += load
        total_time += time - service_time

    print("\nTotal distance of all routes: {}m".format(total_distance))
    print("Total load of all routes: {}".format(total_load))
    print("Total time of all routes: {}".format(total_time))
    print("Total number of vehicles used: {}".format(num_vehicles))





'''if __name__ == '__main__':
    distance_matrix = [
        [0, 1, 2, 3],
        [1, 0, 4, 5],
        [2, 4, 0, 6],
        [3, 5, 6, 0]
    ]
    demands = [1, 0, 2, 3]
    depot_idx = 1
    vehicle_capacity = 4

    solution = genetic_algorithm(distance_matrix, demands, depot_idx, vehicle_capacity)
    print(f"Solution: {solution}")
    print_solution(solution, distance_matrix, demands,  depot_idx)'''

'if __name__ == \'__main__\':\n    distance_matrix = [\n        [0, 1, 2, 3],\n        [1, 0, 4, 5],\n        [2, 4, 0, 6],\n        [3, 5, 6, 0]\n    ]\n    demands = [1, 0, 2, 3]\n    depot_idx = 1\n    vehicle_capacity = 4\n\n    solution = genetic_algorithm(distance_matrix, demands, depot_idx, vehicle_capacity)\n    print(f"Solution: {solution}")\n    print_solution(solution, distance_matrix, demands,  depot_idx)'

In [ ]:
depot_idx = 50
vehicle_capacity = 160
service_time = 10
time_per_distance = 1
max_duration = 200
solution = genetic_algorithm(dist_matrix, demand, depot_idx, vehicle_capacity, service_time, time_per_distance, max_duration)
print(f"Solution: {solution}")
print_solution(solution, dist_matrix, demand, depot_idx, service_time, time_per_distance)

Solution: ([[50, 0, 21, 27, 30, 25, 6, 22, 5, 50], [50, 16, 14, 32, 44, 41, 43, 36, 11, 50], [50, 17, 12, 40, 39, 18, 3, 46, 50], [50, 4, 48, 9, 38, 29, 33, 20, 49, 8, 37, 50], [50, 31, 1, 19, 2, 35, 34, 28, 15, 10, 45, 50], [50, 26, 47, 7, 42, 23, 24, 13, 50]], 6)
Route for vehicle 0:
 50 Load(7.0) -> 0 Load(15.0) -> 21 Load(29.0) -> 27 Load(40.0) -> 30 Load(47.0) -> 25 Load(66.0) -> 6 Load(82.0) -> 22 Load(97.0) -> 50 Load(97.0)
Distance of the route: 87.2330149201813m
Load of the route: 97.0
Time of the route: 157.23301492018132
Route for vehicle 1:
 50 Load(3.0) -> 16 Load(13.0) -> 14 Load(36.0) -> 32 Load(46.0) -> 44 Load(59.0) -> 41 Load(75.0) -> 43 Load(84.0) -> 36 Load(113.0) -> 50 Load(113.0)
Distance of the route: 101.01420794200101m
Load of the route: 113.0
Time of the route: 171.01420794200098
Route for vehicle 2:
 50 Load(41.0) -> 17 Load(64.0) -> 12 Load(91.0) -> 40 Load(98.0) -> 39 Load(107.0) -> 18 Load(116.0) -> 3 Load(141.0) -> 50 Load(141.0)
Distance of the route: 93

In [ ]:
print_solution(solution, dist_matrix, demand, depot_idx, service_time, time_per_distance)